<a href="https://colab.research.google.com/github/Kishore-C-Dev/ML-AI/blob/main/generate_datasets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import requests
import torch
import gradio as gr
from google.colab import drive,userdata
from huggingface_hub import login
from transformers import AutoTokenizer,AutoModelForCausalLM,TextStreamer,BitsAndBytesConfig


In [ ]:
!pip install -q gradio




   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 MB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.4/321.4 kB 26.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 70.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 5.3 MB/s eta 0:00:00


In [ ]:
HF_TOKEN = userdata.get("HF_TOKEN")
login(HF_TOKEN)
model_name = "meta-llama/Meta-Llama-3.1-8B-Instruct"
quant_config=BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4_bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4"
)


Unused kwargs: ['bnb_4_bit_use_double_quant']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.


In [ ]:
system_prompt= "You are helpful assistant to generate datasets \
  Topic: Bank Customer Assistant  \
  Instruction : how can i close my account \
  Resposne : You can walk into branch and request for clouser of your account a bank represntative will help you \
  Instruction: How can i change my debit card pin \
  Resposne: You can go an ATM and click on change pin optiona and follow instruction or you can change it in App \
  Instruction: How can i change my debit card pin \
  Resposne: You can go an ATM and click on change pin optiona and follow instruction or you can change it in App \
  Instruction: How can i see my recent transaction \
  Resposne: You can check in bank App or logging into bank website and clicking into statements tab \
  Return 10 examples each time.Do not repeat the provided examples. \
  Return the dataset in JSON format to train the model. \
  "

messages = [{"role": "system", "content": system_prompt},
            {"role": "user", "content": "please generate a dataset for bank customer assistant" }
            ]


In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token=tokenizer.eos_token


In [ ]:
inputs=tokenizer.apply_chat_template(messages,return_tensors="pt").to("cuda")
streamer=TextStreamer(tokenizer)

In [ ]:
print(inputs)

tensor([[128000, 128006,   9125, 128007,    271,  38766,   1303,  33025,   2696,
             25,   6790,    220,   2366,     18,    198,  15724,   2696,     25,
            220,   1627,  10263,    220,   2366,     19,    271,   2675,    527,
          11190,  18328,    311,   7068,  30525,    256,  34011,     25,   8715,
          12557,  22103,    262,  30151,    551,   1268,    649,    602,   3345,
            856,   2759,    256,   1838,    981,    818,    551,   1472,    649,
           4321,   1139,   9046,    323,   1715,    369,   1206,  89105,    315,
            701,   2759,    264,   6201,   2109,    417,    406,   1413,    690,
           1520,    499,    256,  30151,     25,   2650,    649,    602,   2349,
            856,  46453,   3786,   9160,    256,   1838,    981,    818,     25,
           1472,    649,    733,    459,  47135,    323,   4299,    389,   2349,
           9160,   3072,     64,    323,   1833,   7754,    477,    499,    649,
           2349,    433,    

In [ ]:
model=AutoModelForCausalLM.from_pretrained(model_name,device_map="auto",quantization_config=quant_config)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

In [ ]:
outputs=model.generate(inputs=inputs,max_new_tokens=2000,streamer=streamer)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

You are helpful assistant to generate datasets   Topic: Bank Customer Assistant    Instruction : how can i close my account   Resposne : You can walk into branch and request for clouser of your account a bank represntative will help you   Instruction: How can i change my debit card pin   Resposne: You can go an ATM and click on change pin optiona and follow instruction or you can change it in App   Instruction: How can i change my debit card pin   Resposne: You can go an ATM and click on change pin optiona and follow instruction or you can change it in App   Instruction: How can i see my recent transaction   Resposne: You can check in bank App or logging into bank website and clicking into statements tab   Return 10 examples each time.Do not repeat the provided examples.   Return the dataset in JSON format to train the model.<|eot_id|><|start_header_id|>user<|end_

In [ ]:
default_topic = "Talking to a (5-8) years old and teaching them manners."
default_number_of_data = 10
default_multi_shot_examples = [
    {
        "instruction": "Why do I have to say please when I want something?",
        "response": "Because it’s like magic! It shows you’re nice, and people want to help you more."
    },
    {
        "instruction": "What should I say if someone gives me a toy?",
        "response": "You say, 'Thank you!' because it makes them happy you liked it."
    },
    {
        "instruction": "why should I listen to my parents?",
        "response": "Because parents want the best for you and they love you the most."
    }
]

In [ ]:
def generate_dataset(topic, number_of_data, inst1, resp1, inst2, resp2, inst3, resp3):
    # Convert user inputs into multi-shot examples
    multi_shot_examples = [
        {"instruction": inst1, "response": resp1},
        {"instruction": inst2, "response": resp2},
        {"instruction": inst3, "response": resp3}
    ]

    # System prompt
    system_prompt = f"""
    You are a helpful assistant whose main purpose is to generate datasets.
    Topic: {topic}
    Return the dataset in JSON format. Use examples with simple, fun, and easy-to-understand instructions for kids.
    Include the following examples: {multi_shot_examples}
    Return {number_of_data} examples each time.
    Do not repeat the provided examples.
    """

    # Example Messages
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": f"Please generate my dataset for {topic}"}
    ]

    inputs=tokenizer.apply_chat_template(messages,return_tensors="pt").to("cuda")
    outputs=model.generate(inputs=inputs,max_new_tokens=2000,streamer=streamer)
    return tokenizer.decode(outputs[0], skip_special_tokens=True,skip_prompt=True)


In [ ]:
def gradio_interface(topic, number_of_data, inst1, resp1, inst2, resp2, inst3, resp3):
    return generate_dataset(topic, number_of_data, inst1, resp1, inst2, resp2, inst3, resp3)

In [ ]:
gr_interface = gr.Interface(
    fn=gradio_interface,
    inputs=[
        gr.Textbox(label="Topic", value=default_topic, lines=2),
        gr.Number(label="Number of Examples", value=default_number_of_data, precision=0),
        gr.Textbox(label="Instruction 1", value=default_multi_shot_examples[0]["instruction"]),
        gr.Textbox(label="Response 1", value=default_multi_shot_examples[0]["response"]),
        gr.Textbox(label="Instruction 2", value=default_multi_shot_examples[1]["instruction"]),
        gr.Textbox(label="Response 2", value=default_multi_shot_examples[1]["response"]),
        gr.Textbox(label="Instruction 3", value=default_multi_shot_examples[2]["instruction"]),
        gr.Textbox(label="Response 3", value=default_multi_shot_examples[2]["response"]),
    ],
    outputs=gr.Textbox(label="Generated Dataset")
)

In [ ]:
gr_interface.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://317a5352315cb4f3c1.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
